In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv('C:/Users/Greta/Downloads/responses.csv')
#This allows me to access the dataset
teens = df['Age'] <= 20
df = df[teens]
df = df.dropna()
#limiting the analysis to teenagers

In [30]:
#establishing categories
happy = df['Happiness in life'] >= 4
unhappy = df['Happiness in life'] <= 3
df_happy = df[happy]
df_unhappy = df[unhappy]
upbeat = df['Slow songs or fast songs'] >= 4
downbeat = df['Slow songs or fast songs'] <= 3
df_upbeat = df[upbeat]
df_downbeat = df[downbeat]
#creating lists that will allow me to complete the first comparison of means
happymusic = list(df_happy['Music'])
unhappymusic = list(df_happy['Music'])

In [29]:
from scipy import stats
#distinguishing the happiness levels of those who listen to upbeat music from that of those who listen to downbeat music
happinessupbeat = list(df_upbeat['Happiness in life'])
happinessdownbeat = list(df_downbeat['Happiness in life'])
#establishing a function that will allow me to calculate the confidence interval of a mean
def CI_mean(data):
    se = np.std(data)/np.sqrt(len(data))
    low = np.mean(data)-1.96*se
    up = np.mean(data)+1.96*se
    print("Average is:", np.mean(data))
    print("95% CI of mean is: ({}, {})".format(low,up))
#calculating mean confidence intervals
CI_mean(happinessupbeat)
CI_mean(happinessdownbeat)
#performing a two-value t-test on the two variables established earlier
t_val, p_val = stats.ttest_ind(happinessupbeat, happinessdownbeat, equal_var=True)
print("Test statistic:", t_val)
print("p-value:", p_val)


Average is: 3.725806451612903
95% CI of mean is: (3.570390028057287, 3.881222875168519)
Average is: 3.6930693069306932
95% CI of mean is: (3.606173576348707, 3.7799650375126794)
Test statistic: 0.3802852373342794
p-value: 0.7039236752586588


In [34]:
#creating the "mental health index" category by changing the scale on certain values and then combining them
def change(row):
    return 6 - row['Mood swings']
df["Mood swings 2"] = df.apply(change, axis = 1)
def change2(row):
    return 6 - row['Loneliness']
df["Loneliness 2"] = df.apply(change2, axis = 1)
def change3(row):
    return 6 - row['Getting angry']
df["Getting angry 2"] = df.apply(change3, axis = 1)
def mhi(row):
    return row["Happiness in life"] + row["Mood swings 2"] + row["Loneliness 2"] + row["Getting angry 2"] + row["Energy levels"]
df["Mental health index"] = df.apply(mhi, axis = 1)
#re-establishing the criteria for upbeat and downbeat songs
upbeat = df['Slow songs or fast songs'] >= 4
downbeat = df['Slow songs or fast songs'] <= 3
df_upbeat = df[upbeat]
df_downbeat = df[downbeat]
mhupbeat = list(df_upbeat['Mental health index'])
mhdownbeat = list(df_downbeat['Mental health index'])
#calculating confidence intervals for both variables
CI_mean(mhupbeat)
CI_mean(mhdownbeat)
#performing a t-test on both variables
t_val, p_val = stats.ttest_ind(mhupbeat, mhdownbeat, equal_var=True)
print("Test statistic:", t_val)
print("p-value:", p_val)

Average is: 16.411290322580644
95% CI of mean is: (15.788583491919457, 17.03399715324183)
Average is: 16.0
95% CI of mean is: (15.629871284200618, 16.370128715799382)
Test statistic: 1.144897027736513
p-value: 0.25289614831637175


In [35]:
!pip install statsmodels --user
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
#preparing the two variables for a regression analysis
x_vals = df['Slow songs or fast songs'].values
y_vals = df['Mental health index']
#performing a regression analysis
reg_model = OLS(y_vals, sm.add_constant(x_vals)).fit()
display(reg_model.summary())


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     Mental health index   R-squared:                       0.003
Model:                             OLS   Adj. R-squared:                  0.000
Method:                  Least Squares   F-statistic:                     1.205
Date:                 Wed, 23 Dec 2020   Prob (F-statistic):              0.273
Time:                         22:39:57   Log-Likelihood:                -1123.7
No. Observations:                  427   AIC:                             2251.
Df Residuals:                      425   BIC:                             2259.
Df Model:                            1                                         
Covariance Type:             nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         15.3533      0.717     21.419      0.000      13.944      16.762
x1             0.2355      0.215      1.098      0.273      -0.186       0.657
==============================================================================
Omnibus:                       12.485   Durbin-Watson:                   1.938
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               12.857
Skew:                          -0.386   Prob(JB):                      0.00162
Kurtosis:                       3.355   Cond. No.                         15.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [36]:
#establishing a new column that recognizes music taste as a binary variable
def music(row):
    if row['Pop'] > row['Rock']:
        return 'Pop'
    else:
        return 'Rock'
df['Music'] = df.apply(music, axis = 1)
#establishing a new column that recognizes happiness as a binary variable
def happy(row):
    if row['Happiness in life'] >= 4:
        return 'Happy'
    else:
        return 'Unhappy'
df['Happy'] = df.apply(happy, axis = 1)
#establishing a new column that recognizes mental health as a binary variable
def healthy(row):
    if row['Mental health index'] >= 18:
        return 'Healthy'
    else:
        return 'Unhealthy'
df['Healthy'] = df.apply(healthy, axis = 1)
#creating a contingency table for the first chi-squared test
cont1 = pd.crosstab(df['Happy'],df['Music'], margins = True)
print(cont1)
#performing the first chi-squared test
chi2, p, dof, expected = stats.chi2_contingency(cont1)
print("chi2:", chi2)
print("p:", p)
print("dof:", dof)
print("expected:", expected)
#creating a contingency table for the second chi-squared test
cont2 = pd.crosstab(df['Healthy'],df['Music'], margins = True)
print(cont2)
#performing the second chi-squared test
chi2, p, dof, expected = stats.chi2_contingency(cont2)
print("chi2:", chi2)
print("p:", p)
print("dof:", dof)
print("expected:", expected)
#creating a function that will allow me to obtain the confidence interval for population proportions
def CI_prop(s, n):
    p = s / n
    se2 = p * (1-p) / n
    se = np.sqrt(se2)
    low = p - 1.96 * se
    up = p + 1.96 * se
    print('Sample proportion is:', p)
    print('95% CI is: ({}, {})'.format(low, up))
#calculating population proportions for all binary variables used here
CI_prop(279, 427)
CI_prop(154, 427)
CI_prop(123, 427)
CI_prop(304, 427)

Music    Pop  Rock  All
Happy                  
Happy     89   190  279
Unhappy   34   114  148
All      123   304  427
chi2: 3.7574561678121854
p: 0.43982448699528265
dof: 4
expected: [[ 80.3676815 198.6323185 279.       ]
 [ 42.6323185 105.3676815 148.       ]
 [123.        304.        427.       ]]
Music      Pop  Rock  All
Healthy                  
Healthy     49   105  154
Unhealthy   74   199  273
All        123   304  427
chi2: 1.0659449174773312
p: 0.8996370951310536
dof: 4
expected: [[ 44.36065574 109.63934426 154.        ]
 [ 78.63934426 194.36065574 273.        ]
 [123.         304.         427.        ]]
Sample proportion is: 0.6533957845433255
95% CI is: (0.6082572746449765, 0.6985342944416745)
Sample proportion is: 0.36065573770491804
95% CI is: (0.315109139822376, 0.4062023355874601)
Sample proportion is: 0.28805620608899296
95% CI is: (0.24510220672776867, 0.33101020545021725)
Sample proportion is: 0.711943793911007
95% CI is: (0.6689897945497827, 0.7548977932722314)
